In [105]:
pip install -r requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [106]:
from flask import Flask, request, jsonify, render_template
from flask_cors import CORS, cross_origin
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as urReq
from selenium import webdriver
import requests, mysql, pymongo, time, os

In [107]:
DRIVER_PATH = r'chromedriver.exe'

In [108]:
# searchText = request.form['searchtext'].replace(" ","+")
driver = webdriver.Chrome()
searchURL = "https://www.youtube.com/results?search_query=krish+naik"
driver.get(searchURL)

# requestURL = requests.get(searchURL)
# youtubePage = requestURL.read()
# requestURL.close()

In [109]:
# youtubePage
content = driver.page_source.encode('utf-8').strip()
soup = bs(content,'html.parser')
findChannelSection = soup.find_all('div',id="content-section")

In [110]:
# from selenium.webdriver.common.by import By
# abc = driver.find_element(By.CSS_SELECTOR,'div.content-section')

In [111]:
# abc.

In [112]:
print(len(findChannelSection))

1


In [113]:
type(findChannelSection)

bs4.element.ResultSet

In [114]:
for tag in findChannelSection:
    channelLink = tag.div.a['href']
    print(channelLink)

/user/krishnaik06


In [115]:
channelEmbeddedURL = "https://www.youtube.com{}/videos?view=0&sort=p&flow=grid".format(channelLink)

In [116]:
channelEmbeddedURL

'https://www.youtube.com/user/krishnaik06/videos?view=0&sort=p&flow=grid'

In [117]:
driver.get(channelEmbeddedURL)
# driver.send_keys(Keys.ARROW_DOWN)
driver.execute_script("window.scrollTo(0,2000);")
time.sleep(1)

In [118]:
findChannelPage = driver.page_source.encode('utf-8').strip()


In [119]:
pageSoup = bs(findChannelPage,'lxml')

In [120]:
titles = pageSoup.findAll('a',id="video-title")
views = pageSoup.findAll('span',class_="style-scope ytd-grid-video-renderer")

channel_details = []


In [128]:
len(titles)

60

In [122]:
thumbnail = pageSoup.findAll('img',class_="style-scope yt-img-shadow")

In [123]:
len(thumbnail)

69

In [124]:
thumb_list = []
for thumb in thumbnail[7:]:
    thumb_list.append(thumb.get('src'))

In [125]:
thumb_list

['https://i.ytimg.com/vi/k2P_pHQDlp0/hqdefault.jpg?sqp=-oaymwEcCPYBEIoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLBaMLAnbB190v0LBkb5p3ET9k73Nw',
 'https://i.ytimg.com/vi/csG_qfOTvxw/hqdefault.jpg?sqp=-oaymwEcCPYBEIoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLC3ApsU1Y6EV-S5mLV8vJ8OsJ5roQ',
 'https://i.ytimg.com/vi/bPrmA1SEN2k/hqdefault.jpg?sqp=-oaymwEcCPYBEIoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLBB8olHjCipKtCvPjUGWZDsdWqTUA',
 'https://i.ytimg.com/vi/M-UpXWGxZe4/hqdefault.jpg?sqp=-oaymwEcCPYBEIoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLCky1i90z6nTkC_M-MpbZ9Eqw6_3w',
 'https://i.ytimg.com/vi/fM4qTMfCoak/hqdefault.jpg?sqp=-oaymwEcCPYBEIoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLBAuRaHIENo356042y_sDOa8BKjSg',
 'https://i.ytimg.com/vi/4-kDCP0Bor0/hqdefault.jpg?sqp=-oaymwEcCPYBEIoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLDtsPugct4nLg7CmJqF3eROw_3rRA',
 'https://i.ytimg.com/vi/AuqZ4recf0s/hqdefault.jpg?sqp=-oaymwEcCPYBEIoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLAJsUM9HXUciDJStErXSAgUw

In [127]:
i = 0
j = 0
for title in titles:
    d = {"videoTitle":title.text,"totalViews":views[i].text,"videoURL":"https://www.youtube.com"+title.get('href'),"thumbnail":thumb_list[j]}
    print(d)
    channel_details.append(d)
    i +=2
    j +=1

{'videoTitle': 'AI VS ML VS DL VS Data Science', 'totalViews': '2.2M views', 'videoURL': 'https://www.youtube.com/watch?v=k2P_pHQDlp0', 'thumbnail': 'https://i.ytimg.com/vi/k2P_pHQDlp0/hqdefault.jpg?sqp=-oaymwEcCPYBEIoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLBaMLAnbB190v0LBkb5p3ET9k73Nw'}
{'videoTitle': 'How To Learn Data Science Smartly?', 'totalViews': '1.4M views', 'videoURL': 'https://www.youtube.com/watch?v=csG_qfOTvxw', 'thumbnail': 'https://i.ytimg.com/vi/csG_qfOTvxw/hqdefault.jpg?sqp=-oaymwEcCPYBEIoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLC3ApsU1Y6EV-S5mLV8vJ8OsJ5roQ'}
{'videoTitle': 'Complete Road Map To Be Expert In Python- Follow My Way', 'totalViews': '1.3M views', 'videoURL': 'https://www.youtube.com/watch?v=bPrmA1SEN2k', 'thumbnail': 'https://i.ytimg.com/vi/bPrmA1SEN2k/hqdefault.jpg?sqp=-oaymwEcCPYBEIoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLBB8olHjCipKtCvPjUGWZDsdWqTUA'}
{'videoTitle': 'OTT Platform For Education OneNeuron- Education As A Service', 'totalViews': '7